In [40]:
CHROMA_PATH = "msd_chroma"

In [41]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [42]:
def get_embeddings():
   model_kwargs = {'device': 'cuda'}
   embeddings_hf = HuggingFaceEmbeddings(
       model_name='intfloat/multilingual-e5-large',
       model_kwargs=model_kwargs
   )
   return embeddings_hf

In [43]:
PROMPT_TEMPLATE = """
Вы — медицинский ассистент для врачей-специалистов. Вы предоставляете информацию исключительно на основе статей из MSD справочника и клинических рекомендаций по лечению определенных заболеваний. Ваши ответы должны быть точными, основанными на этих источниках, и представлены в понятной и сжатой форме. Если информация отсутствует в указанных источниках, сообщите об этом и избегайте предположений.

Ответь на вопрос базируясь только на этом контексте: 

{context}

---

Вопрос: {question}
"""

In [44]:
import argparse
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_community.vectorstores import Chroma


In [45]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embeddings())


In [46]:
query_text='пояснично крестцовый радикулопатия?'
results = db.similarity_search_with_relevance_scores(query_text, k=5)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Нет фрагментов текста, на которые можно опираться для ответа.")

In [47]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(f"Полученный промпт {prompt}")

Полученный промпт Human: 
Вы — медицинский ассистент для врачей-специалистов. Вы предоставляете информацию исключительно на основе статей из MSD справочника и клинических рекомендаций по лечению определенных заболеваний. Ваши ответы должны быть точными, основанными на этих источниках, и представлены в понятной и сжатой форме. Если информация отсутствует в указанных источниках, сообщите об этом и избегайте предположений.

Ответь на вопрос базируясь только на этом контексте: 

этиологии.
Хотя, серьезные спинальные и экстраспинальные заболевания являются редкими причинами, симптомы "красных флагов" часто указывают на
потребность в обследовании.
Оценка функции спинного мозга при физикальном осмотре включает исследования функции крестцового нерва (например, тонуса прямой
кишки, анального рефлекса, бульбокавернозного рефлекса), а также исследованием коленного и ахиллова рефлексов и двигательных функций.
Пациенты с сегментарным неврологическим дефицитом, который может быть связан с компрессие

In [48]:
model = ChatOpenAI(temperature=0.7, base_url="http://localhost:1234/v1", api_key="not-needed")
response_text = model.predict(prompt)


sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Ответ: {response_text}\nДанные взяты из: {sources}"
print(formatted_response)

Ответ: Пояснично-крестцовая радикулопатия - это тип радикулопатии, который возникает из-за компрессии или повреждения нервных корешков в спинном мозге, расположенных на уровне поясницы и крестца.

Характерные симптомы пояснично-крестцовой радикулопатии включают:

* Боль в ногах и ягодицах
* Поражение чувствительности и парестезии на нижней части тела, обычно ниже пояса
* Слабость и атрофия мышц бедер и голеней
* Утрата рефлексов на нижних конечностях

Пояснично-крестцовую радикулопатию можно вызвать множество факторов, включая:

* Грыжевой межпозвоночный диск
* Ревматоидный артрит или остеоартрит
* Карциноматозный менингит
* Инфекции, такие как туберкулез или гистоплазмоз
* Опоясывающий лишай
* Другие причины, такие как травма или метаболические нарушения.

Диагноз можно поставить на основе клинического обследования и результатов нейровизуализации, таких как МРТ или КТ. Лечение зависит от причины радикулопатии и может включать симптоматическую терапию нестероидными противовоспалительны